*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'0_GlobalData.xlsx'), read)
readSets(db)

C:\Users\xnh825\Anaconda3\envs\EEotGT\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\xnh825\Documents\GitHub\EnergyEconomicsE2022\py\readDatabase.py:63: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  var = pd.DataFrame(pd_sheet.iloc[1:,1:].values, index = pd.Index(pd_sheet.iloc[1:,0],name=domains[1]), columns = pd.Index(pd_sheet.iloc[0,1:], name = domains[2])).stack()


# The ```mBasicInt``` model

### **The model**

The class specifies the relatively simple linear electricity system model specified as:
<a id='simplemodel'>
$$\begin{align} \tag{1}
    \max \text{Welfare} &= \sum_h\left(u\cdot D_h-\sum_{id}mc_{id}\cdot E_{id,h}\right) \\ 
               D_h &= \sum_{id} E_{id,h} \\ 
               D_h &\in [0, z_h\cdot \text{Load}], \\
               E_{id,h}&\in[0,q_{id,h}].
\end{align}$$

* $u$ is the consumers' marginal willingness to pay for not having the load shedded. 
* $D_h$ is the actual served load in hour $h$ whereas (Load$\cdot z_h$) is the planned level of demand/load in hour $h$; 
* $z_h$ is a measure of the hourly variation in demand. If we make sure that $\sum_h z_h =1$ the 'Load' parameter is a measure of 'yearly'/long run demand. 
* ($z_h\cdot \text{Load} - D_h$) measures the shedded load in a given hour. 
* The boundary constraint $D_h\in[0, z_h\cdot \text{Load}]$ states that demand cannot be negative and that load shedding cannot be negative (we return to some more advanced demand modelling in topic of flexible demand systems).
* The boundary constraint $E_{id,h}\in[0,q_{id,h}]$ states the generation cannot be negative and not exceed a plant-and-hourly specific generation capacity. This generation capacity is generally constant for 'standard' plants, but varies with hours for solar and wind type of plants.

### 1. **Augmented form LP**

To solve the model, the algorithm in ```scipy.optimize.linprog``` requires that we specify the problem in the 'augmented' form that specifically looks as follows:
$$\begin{align} \tag{2}
    &\min_{x}\mbox{ }c^T\cdot x \\ 
    &A_{ub}\times x \leq b_{ub} \\ 
    &A_{eq}\times x  = b_{eq} \\ 
    &l\leq x\leq u,
\end{align}$$
where 
* $c,x,l,u$ are all vectors of the same length $N$, 
* $b_{eq},b_{ub}$ are vectors of lengths $N_{eq},N_{ub}$ respectively,
** and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

You can confirm that to go from the specific model to the augmented form we have the following:
<b id='standardform'> 

#### 1.1. The vector of decision variables $x$:
Consists of a vector of stacked generation choices $E_{id,h}$ for all $id,h$, and the vector of demand levels $D_h$ for each $h$:

$$\begin{align}
x = \begin{pmatrix} E_{id1,1} \\ \vdots \\ E_{idN,H} \\ D_1 \\ \vdots \\ D_H\end{pmatrix}
\end{align}$$

#### 1.2. The vector of cost coefficients $c$:
Note that the model in equations [(1)](#simplemodel) *maximizes* a welfare function, whereas the augmented form model *minimizes* a linear objective function. Thus, to arrive at a augmented form, we need to use the objective:
$$\begin{align}
    \min -\text{Welfare} = \sum_h\left(\sum_{id}\left[mc_{id}\cdot E_{id,h}\right]-u\cdot D_h\right)
\end{align}$$
Consists of (i) a vector of stacked marginal costs components for each $id,h$ that is multiplied on the $E_{id,h}$ variables, and (ii) a vector of $-u$ for each $D_h$ variable:

$$\begin{align}
    c = \begin{pmatrix} \text{repeat}_h\begin{pmatrix} mc_{id1} \\ \vdots \\ mc_{idN} \\ \end{pmatrix} \\ 
    \text{repeat}_h\begin{pmatrix} -u \end{pmatrix} \end{pmatrix},
\end{align}$$
where we use $\text{repeat}_h$ to indicate that a vector is repeated for all $h$ hours.

#### 1.3. The vector of lower bounds $l$:
The lower bounds on all decision variables $(E_{id,h}, D_h)$ is zero. Thus, the $l$ vector is simply a $0$-vector of the same length as $x$ and $c$:
$$\begin{align}
    l = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix} 
\end{align}$$

#### 1.4. The vector of upper bounds $u$:
Consists of (i) hourly generating capacity constraint $q_{id,h}$ and (ii) hourly load 'capacity' constraints: 
$$\begin{align}
    u = \begin{pmatrix} q_{id1,1} \\ \vdots \\ q_{idN,H} \\ \text{Load}\cdot z_1 \\ \vdots \\ \text{Load}\cdot z_H \end{pmatrix} 
\end{align}$$

#### 1.5. Constants in equality constraints $b_{eq}$:
Consists of zeros for each equilibrium constraint (defined for each $h$):
$$\begin{align}
    b_{eq} = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix} 
\end{align}$$

#### 1.6. Coefficients in equality constraints $A_{eq}$:
For each equilibrium constraint (in hour $h$), the matrix $A_{eq}$ consists of a rows of ones for each $id$ in $E_{id,h}$ and $-1$ for the relevant demand variable $D_h$. Thus, the matrix $A_{eq}$ looks something like this:

$$\begin{align}
    A_{eq} = \begin{pmatrix} 1 & \cdots & 1, & 0 & \cdots & 0, & 0 & \cdots & 0, & -1 & \cdots & 0 \\ 
    \vdots & \vdots & \vdots, & \vdots & \vdots & \vdots, & \vdots & \vdots & \vdots, & \vdots & \vdots & \vdots \\
                             0 & \cdots & 0, & 0 & \cdots & 0, & 1 & \cdots & 1, & 0 & \cdots & -1 
    \end{pmatrix} 
\end{align}$$
Note that each row identifies an *hourly* equilibrium constraint. The last columns indicate the coefficients on demand components $D_h$; this is a diagonal matrix with $-1$. The first many columns indicate coefficients on $E_{id,h}$; these consists of blocks of ones for all $id$'s in the relevant hour, $h$.

## 2. Initialize and solve model

The model is initialized with the database as input using: 

In [2]:
m = mBasicInt.mSimple(db)

All models are provided with three main methods: ```self.preSolve```, ```self.initBlocks```, and ```self.postSolve```. The ```preSolve``` and ```postSolve``` methods are used to carry out computations before and after the model is solved, respectively. The ```self.preSolve``` and ```self.postSolve``` are similar to the ones in the [mBasic model](M_mBasic.ipynb). We focus on the ```self.initBlocks``` here. We note that the following statement carries out all three steps:

In [3]:
m.solve()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


C:\Users\xnh825\Anaconda3\envs\EEotGT\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)
C:\Users\xnh825\Anaconda3\envs\EEotGT\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


### ```initBlocks:```

The method adds information that translates the model to an augmented form LP problem. We use the ```lpBlock``` class to help us specify these (see [_Class_lpBlock.ipynb](_Class_lpBlock.ipynb) for help on syntax etc.) 

#### 2.1. The vector of cost coefficients $c$:

As outlined earlier, this consists of two components: Marginal costs $mc_{id}$ repeated for all hours $h$ and minus marginal willing to pay (MWP, $-u$) repeated for all hours $h$.

<img src="snippets/mBasicInt_snippet.png" width="1000" height="400">

We use the ```broadcast``` method to repeat the marginal cost vector for all hours with the resulting coefficient:

In [4]:
lpCompiler.broadcast(m.db['mc'], m.db['h'])

id   h
id1  1    11.872521
     2    11.872521
     3    11.872521
     4    11.872521
id2  1    12.346414
     2    12.346414
     3    12.346414
     4    12.346414
id3  1    12.193476
     2    12.193476
     3    12.193476
     4    12.193476
id4  1     8.393216
     2     8.393216
     3     8.393216
     4     8.393216
id5  1     5.031068
     2     5.031068
     3     5.031068
     4     5.031068
id6  1    23.443656
     2    23.443656
     3    23.443656
     4    23.443656
id7  1          3.0
     2          3.0
     3          3.0
     4          3.0
id8  1          3.0
     2          3.0
     3          3.0
     4          3.0
dtype: object

For the coefficients on 'HourlyDemand', note that we simply add the scalar parameter ```MWP_LoadShedding```:

In [5]:
m.db['MWP_LoadShedding']

25

Recall, though, that we needed to repeat this for all hours. The way we remedy this, is to specify the ```self.globalDomains``` for the model:

<img src="snippets/mBasicInt_globalDomains.png" width="800" height="400">

When we specify a parameter as ```None``` or as a scalar, the program will automatically check for ```self.globalDomains```; we have specified the global domains for ```HourlyDemand```, the scalar parameter will be broadcasted to this domain. In other words, the following will be added to the model:

In [6]:
lpCompiler.broadcast(-m.db['MWP_LoadShedding'], m.db['h'])

h
1   -25
2   -25
3   -25
4   -25
dtype: int64

#### 2.2. The vector of upper bounds $u$:

The upper bounds consists of hourly generation and demand constraints:

<img src="snippets/mBasicInt_snippet2.png" width="1000" height="400">

The two methods ```self.hourlyGeneratingCapacity``` and ```self.hourlyLoad``` are specified elsewhere in the class and returns the symbols:

*Generating capacity:*

In [7]:
m.hourlyGeneratingCapacity

id   h
id8  1       0
     2     5.0
     3      10
     4     1.0
id1  1      10
     2      10
     3      10
     4      10
id2  1      15
     2      15
     3      15
     4      15
id3  1      10
     2      10
     3      10
     4      10
id4  1      30
     2      30
     3      30
     4      30
id5  1       5
     2       5
     3       5
     4       5
id6  1       8
     2       8
     3       8
     4       8
id7  1      35
     2    17.5
     3    8.75
     4    28.0
dtype: object

*Hourly load constraint:*

In [8]:
m.hourlyLoad

h
1    10.0
2    30.0
3    50.0
4    10.0
dtype: object

#### 2.3. Constants in equality constraints $b_{eq}$:
Consists of zeros for each equilibrium constraint (defined for each $h$)

<img src="snippets/mBasicInt_snippet3.png" width="1000" height="400">

Note that we use the same trick here as we did with the $c$ coefficient: In ```self.globalDomains``` we have specified that the ```equilibrium``` constraint is defined over the set ```h_alias``` which is an alias of $h$ (a copy with a separate identifier). Thus, when we add a parameter with ```None``` as here, we automatically broadcast this as a vector of zeros.

#### 2.4. Coefficients in equality constraints $A_{eq}$:

As shown in the previous section, the matrix consists of blocks of ones and zeros for the appropriate $E_{id,h}$ and a diagonal matrix of $-1$ for the $D_h$:

<img src="snippets/mBasicInt_snippet4.png" width="1200" height="400">

Note that the parameter on ```Generation``` is defined as in the ```globalDomains```, but with an added index level called ```h_alias```. The resulting variable is:

In [9]:
lpModels.appIndexWithCopySeries(pd.Series(1, index = m.globalDomains['Generation']), 'h', 'h_alias')

h  id   h_alias
1  id1  1          1
   id2  1          1
   id3  1          1
   id4  1          1
   id5  1          1
   id6  1          1
   id7  1          1
   id8  1          1
2  id1  2          1
   id2  2          1
   id3  2          1
   id4  2          1
   id5  2          1
   id6  2          1
   id7  2          1
   id8  2          1
3  id1  3          1
   id2  3          1
   id3  3          1
   id4  3          1
   id5  3          1
   id6  3          1
   id7  3          1
   id8  3          1
4  id1  4          1
   id2  4          1
   id3  4          1
   id4  4          1
   id5  4          1
   id6  4          1
   id7  4          1
   id8  4          1
dtype: int64

The method ```appIndexWithCopySeries``` copies the index ```h``` with the alias ```h_alias```. Recall that the ```self.globalDomains``` defines the equilibrium constraint over ```h_alias```:

<img src="snippets/mBasicInt_globalDomains.png" width="800" height="400">

Thus, the matrix $A$ above adds $1$'s defined for all  ```h=h_alias```. Note that we do not have to specify all the combinations where the coefficients are zero (when ```h```$\neq$```h_alias```). A similar logic applies to the coefficient on ```HourlyDemand```: The parameter we add is $-1$ for all the combinations where ```h=h_alias```.

In [10]:
lpModels.appIndexWithCopySeries(pd.Series(-1, index = m.globalDomains['HourlyDemand']), 'h', 'h_alias')

h  h_alias
1  1         -1
2  2         -1
3  3         -1
4  4         -1
dtype: int64

You can confirm that the final compiled model looks as we discussed in the previous section by printing:

In [11]:
m.blocks.lp_A_eq

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0., -1.]])